In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
web_instance = Website("https://vishal-lingineni.medium.com/effortlessly-sync-your-google-colab-or-jupyter-notebook-data-projects-with-github-a-step-by-step-61fb5019abc5")
print(web_instance.title)
print(web_instance.text)

In [ ]:
user_prompt = "Extact the information and list down the process in bullet points. related to uploading a jupyter project to github. use the below text for reference \n\n"
user_prompt+=web_instance.text
messages = [
    {"role": "user", "content": user_prompt}
]
print(messages)
print(user_prompt)

In [ ]:
#loading constants:
MODEL = "llama3.2"
OLLAMA_API="http://localhost:11434/api/chat"
HEADERS={"Content-Type": "application/json"}

In [ ]:
#load ollama
!ollama pull llama3.2

In [ ]:
#calling ollama using requests library
payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

In [ ]:
# directly calling ollama lib
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

In [ ]:
# calling ol
from openai import OpenAI

ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key = 'ollama')

response_ollama_via_openai = ollama_via_openai.chat.completions.create(model=MODEL, messages=messages)

print(response_ollama_via_openai.choices[0].message.content)